In [1]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [2]:
from selenium import webdriver
import time

In [3]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [4]:
import os
print(os.getcwd())

C:\Users\bitcamp\pandas_prep


In [5]:
driver = webdriver.Chrome('./chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [6]:
driver.find_element_by_id("electionType1").click()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.114)


In [7]:
driver = webdriver.Chrome('./chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")
driver.find_element_by_id("electionType1").click()

In [8]:
driver.find_element_by_id("electionName").send_keys("제19대")

In [9]:
driver.find_element_by_id("electionCode").send_keys("대통령선거")

In [10]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [11]:
import re

def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH,make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()

In [13]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(get_num(each[2]))
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

In [14]:
election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],    
                       'ahn' : [] }

In [15]:
for each_sido in sido_names_values:
    move_sido(each_sido)

In [16]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    print(type(table))

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [17]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

TypeError: expected string or bytes-like object

In [21]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    print(df)
    # print(df.columns)
    print(type(df))

[    구시군명     선거인수      투표수     후보자별 득표수(득표율)                    \
    구시군명     선거인수      투표수         더불어민주당문재인          자유한국당홍준표   
0     합계  8382999  6590646  2,781,345(42.34)  1,365,285(20.78)   
1    종로구   133769   102566     42,512(41.59)     22,325(21.84)   
2     중구   109836    82852     34,062(41.23)     17,901(21.67)   
3    용산구   197962   148157     58,081(39.33)     35,230(23.85)   
4    성동구   259009   203175     86,686(42.80)     40,566(20.03)   
5    광진구   305172   240030    105,512(44.10)     46,368(19.38)   
6   동대문구   304972   236092     98,958(42.06)     51,631(21.94)   
7    중랑구   351986   265706    111,450(42.09)     56,545(21.35)   
8    성북구   378024   295866    129,263(43.82)     57,584(19.52)   
9    강북구   281434   210614     89,645(42.70)     42,268(20.13)   
10   도봉구   293679   229233     94,898(41.55)     47,461(20.78)   
11   노원구   464422   374213    158,167(42.41)     67,590(18.12)   
12   은평구   413086   317275    140,180(44.34)     60,109(19.01)   
13  서대문구 

[  구시군명     선거인수      투표수   후보자별 득표수(득표율)                                  \
  구시군명     선거인수      투표수       더불어민주당문재인        자유한국당홍준표         국민의당안철수   
0   합계  2043276  1581347  342,620(21.76)  714,205(45.36)  235,757(14.97)   
1   중구    68857    52180   11,477(22.09)   24,698(47.54)    7,170(13.80)   
2   동구   294274   226510   47,350(20.99)  100,166(44.41)   31,351(13.90)   
3   서구   173359   127659   22,004(17.32)   68,782(54.15)   17,808(14.02)   
4   남구   136206   100059   19,862(19.94)   50,304(50.51)   13,978(14.03)   
5   북구   355739   276725   62,526(22.69)  121,368(44.05)   42,369(15.37)   
6  수성구   356388   285528   64,922(22.82)  123,074(43.26)   43,698(15.36)   
7  달서구   479081   374766   82,726(22.17)  165,781(44.43)   57,956(15.53)   
8  달성군   179372   137920   31,753(23.13)   60,032(43.73)   21,427(15.60)   

                                                                               \
          바른정당유승민        정의당심상정      새누리당조원진   경제애국당오영국  국민대통합당장성민  늘푸른한국당이재오   


[  구시군명    선거인수     투표수   후보자별 득표수(득표율)                                  \
  구시군명    선거인수     투표수       더불어민주당문재인        자유한국당홍준표         국민의당안철수   
0   합계  941093  744960  282,794(38.14)  203,602(27.46)  128,520(17.33)   
1   중구  198415  157051   56,657(36.25)   47,625(30.47)   26,458(16.93)   
2   남구  276867  219569   80,662(36.89)   63,331(28.96)   39,040(17.85)   
3   동구  138379  109054   44,931(41.42)   23,701(21.85)   19,290(17.78)   
4   북구  151453  122933   52,058(42.53)   26,596(21.73)   20,115(16.43)   
5  울주군  175979  136353   48,486(35.75)   42,349(31.22)   23,617(17.41)   

                                                                            \
        바른정당유승민         정의당심상정    새누리당조원진   경제애국당오영국  국민대통합당장성민  늘푸른한국당이재오   
0  60,289(8.13)   62,187(8.38)  829(0.11)  128(0.01)  575(0.07)  240(0.03)   
1  12,399(7.93)   12,307(7.87)  183(0.11)   21(0.01)  130(0.08)   53(0.03)   
2  18,536(8.47)   16,078(7.35)  259(0.11)   29(0.01)  128(0.05)   66(0.03)   
3   8,337(7.68) 

[   구시군명     선거인수     투표수   후보자별 득표수(득표율)                                  \
   구시군명     선거인수     투표수       더불어민주당문재인        자유한국당홍준표         국민의당안철수   
0    합계  1287173  955885  324,768(34.16)  284,909(29.97)  206,840(21.75)   
1   춘천시   229286  175879   66,874(38.18)   43,494(24.83)   38,684(22.09)   
2   원주시   273858  204233   77,495(38.10)   50,750(24.95)   45,776(22.50)   
3   강릉시   178461  132395   41,565(31.55)   44,421(33.72)   26,895(20.41)   
4   동해시    76059   55542   17,184(31.09)   19,009(34.39)   11,345(20.52)   
5   삼척시    59080   44097   13,496(30.81)   15,971(36.46)    8,185(18.68)   
6   태백시    39001   28101    9,557(34.19)   10,046(35.94)    4,971(17.78)   
7   정선군    33408   23859    8,270(34.88)    8,323(35.10)    4,489(18.93)   
8   속초시    67471   46985   15,045(32.16)   13,883(29.67)   11,031(23.58)   
9   고성군    25637   18692    5,664(30.58)    6,511(35.15)    3,964(21.40)   
10  양양군    23853   17853    5,017(28.29)    6,687(37.70)    3,868(21.81)   
11  인제군    

[      구시군명     선거인수      투표수   후보자별 득표수(득표율)                                  \
      구시군명     선거인수      투표수       더불어민주당문재인        자유한국당홍준표         국민의당안철수   
0       합계  1711912  1240204  476,661(38.62)  306,614(24.84)  290,216(23.51)   
1   천안시서북구   285471   212329   94,202(44.53)   37,293(17.62)   47,891(22.63)   
2   천안시동남구   207268   151132   59,695(39.67)   34,869(23.17)   34,933(23.21)   
3      공주시    93453    67018   25,036(37.58)   18,449(27.69)   15,814(23.73)   
4      보령시    87274    62825   20,674(33.13)   19,453(31.18)   15,428(24.72)   
5      아산시   236407   173795   73,461(42.45)   36,741(21.23)   38,368(22.17)   
6      서산시   137217    98859   37,376(37.99)   24,572(24.97)   23,237(23.62)   
7      태안군    55379    38740   12,881(33.48)   12,666(32.92)    8,940(23.23)   
8      금산군    46637    32968   11,182(34.17)    9,416(28.78)    8,787(26.85)   
9      논산시   103557    72434   28,265(39.23)   16,948(23.52)   18,844(26.15)   
10     계룡시    32653    26530    9,566(3

[   구시군명     선거인수      투표수   후보자별 득표수(득표율)                                \
   구시군명     선거인수      투표수       더불어민주당문재인      자유한국당홍준표         국민의당안철수   
0    합계  1572838  1238738  737,921(59.87)  30,221(2.45)  378,179(30.68)   
1   목포시   187620   145476   77,896(53.73)   2,584(1.78)   53,303(36.77)   
2   여수시   235605   185487  117,586(63.63)   4,281(2.31)   50,228(27.18)   
3   순천시   223756   181451  122,595(67.81)   4,525(2.50)   40,429(22.36)   
4   나주시    89312    71599   43,967(61.76)   1,448(2.03)   21,011(29.51)   
5   광양시   120767    96384   63,544(66.15)   4,100(4.26)   20,080(20.90)   
6   담양군    41434    33020   19,961(60.80)     698(2.12)   10,017(30.51)   
7   장성군    39083    30670   17,378(56.93)     851(2.78)   10,167(33.31)   
8   곡성군    26671    21247   12,152(57.73)     624(2.96)    6,850(32.54)   
9   구례군    23487    18924   11,873(63.11)     628(3.33)    5,188(27.57)   
10  고흥군    60101    47036   26,882(57.66)   1,349(2.89)   15,614(33.49)   
11  보성군    38902    2996

[        구시군명     선거인수      투표수   후보자별 득표수(득표율)                  \
        구시군명     선거인수      투표수       더불어민주당문재인        자유한국당홍준표   
0         합계  2744633  2135055  779,731(36.73)  790,491(37.24)   
1     창원시의창구   207375   164047   60,757(37.22)   56,887(34.85)   
2     창원시성산구   183547   153327   63,717(41.74)   42,052(27.54)   
3   창원시마산합포구   153651   119281   35,592(29.99)   54,488(45.91)   
4   창원시마산회원구   172306   136757   45,014(33.07)   56,340(41.39)   
5     창원시진해구   146938   114779   41,249(36.11)   40,049(35.06)   
6        진주시   281105   222813   73,929(33.35)   93,751(42.30)   
7        통영시   111279    82855   25,477(30.94)   36,128(43.87)   
8        고성군    47369    34603    9,848(28.67)   16,797(48.91)   
9        사천시    94377    71555   22,370(31.47)   32,475(45.69)   
10       김해시   414018   318253  147,972(46.72)   82,880(26.17)   
11       밀양시    93568    70687   20,842(29.68)   32,394(46.14)   
12       거제시   196936   150230   68,291(45.71)   38,775(25.95)   
13       

In [23]:
move_sido('서울특별시')
    
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
    
df = pd.read_html(str(table))
print(df)

print(type(df))
for i, j in enumerate(df):
    print(f'index: {i}')
    print(f'{i}th element holds: {j}')

[    구시군명     선거인수      투표수     후보자별 득표수(득표율)                    \
    구시군명     선거인수      투표수         더불어민주당문재인          자유한국당홍준표   
0     합계  8382999  6590646  2,781,345(42.34)  1,365,285(20.78)   
1    종로구   133769   102566     42,512(41.59)     22,325(21.84)   
2     중구   109836    82852     34,062(41.23)     17,901(21.67)   
3    용산구   197962   148157     58,081(39.33)     35,230(23.85)   
4    성동구   259009   203175     86,686(42.80)     40,566(20.03)   
5    광진구   305172   240030    105,512(44.10)     46,368(19.38)   
6   동대문구   304972   236092     98,958(42.06)     51,631(21.94)   
7    중랑구   351986   265706    111,450(42.09)     56,545(21.35)   
8    성북구   378024   295866    129,263(43.82)     57,584(19.52)   
9    강북구   281434   210614     89,645(42.70)     42,268(20.13)   
10   도봉구   293679   229233     94,898(41.55)     47,461(20.78)   
11   노원구   464422   374213    158,167(42.41)     67,590(18.12)   
12   은평구   413086   317275    140,180(44.34)     60,109(19.01)   
13  서대문구 

In [24]:
print(len(df))

1


In [25]:
df[0].values

array([['합계', 8382999, 6590646, '2,781,345(42.34)', '1,365,285(20.78)',
        '1,492,767(22.72)', '476,973(7.26)', '425,459(6.47)',
        '9,987(0.15)', '789(0.01)', '3,554(0.05)', '1,938(0.02)',
        '3,416(0.05)', '1,277(0.01)', '2,177(0.03)', '3,950(0.06)',
        6568917, 21729, 1792353],
       ['종로구', 133769, 102566, '42,512(41.59)', '22,325(21.84)',
        '22,313(21.83)', '7,412(7.25)', '7,113(6.95)', '228(0.22)',
        '5(0.00)', '78(0.07)', '31(0.03)', '63(0.06)', '26(0.02)',
        '47(0.04)', '49(0.04)', 102202, 364, 31203],
       ['중구', 109836, 82852, '34,062(41.23)', '17,901(21.67)',
        '19,372(23.45)', '5,874(7.11)', '4,993(6.04)', '158(0.19)',
        '12(0.01)', '53(0.06)', '21(0.02)', '55(0.06)', '18(0.02)',
        '25(0.03)', '51(0.06)', 82595, 257, 26984],
       ['용산구', 197962, 148157, '58,081(39.33)', '35,230(23.85)',
        '32,109(21.74)', '11,825(8.00)', '9,773(6.61)', '299(0.20)',
        '17(0.01)', '68(0.04)', '23(0.01)', '72(0.04)', '32(

In [26]:
df[1]

IndexError: list index out of range

In [27]:
df[0].values[1:]

array([['종로구', 133769, 102566, '42,512(41.59)', '22,325(21.84)',
        '22,313(21.83)', '7,412(7.25)', '7,113(6.95)', '228(0.22)',
        '5(0.00)', '78(0.07)', '31(0.03)', '63(0.06)', '26(0.02)',
        '47(0.04)', '49(0.04)', 102202, 364, 31203],
       ['중구', 109836, 82852, '34,062(41.23)', '17,901(21.67)',
        '19,372(23.45)', '5,874(7.11)', '4,993(6.04)', '158(0.19)',
        '12(0.01)', '53(0.06)', '21(0.02)', '55(0.06)', '18(0.02)',
        '25(0.03)', '51(0.06)', 82595, 257, 26984],
       ['용산구', 197962, 148157, '58,081(39.33)', '35,230(23.85)',
        '32,109(21.74)', '11,825(8.00)', '9,773(6.61)', '299(0.20)',
        '17(0.01)', '68(0.04)', '23(0.01)', '72(0.04)', '32(0.02)',
        '51(0.03)', '80(0.05)', 147660, 497, 49805],
       ['성동구', 259009, 203175, '86,686(42.80)', '40,566(20.03)',
        '45,674(22.55)', '15,859(7.83)', '12,936(6.38)', '304(0.15)',
        '16(0.00)', '94(0.04)', '46(0.02)', '108(0.05)', '41(0.02)',
        '58(0.02)', '124(0.06)', 2025

In [28]:
print(type(df[0]))

<class 'pandas.core.frame.DataFrame'>


In [29]:
df[0].columns

MultiIndex([(         '구시군명',      '구시군명'),
            (         '선거인수',      '선거인수'),
            (          '투표수',       '투표수'),
            ('후보자별 득표수(득표율)', '더불어민주당문재인'),
            ('후보자별 득표수(득표율)',  '자유한국당홍준표'),
            ('후보자별 득표수(득표율)',   '국민의당안철수'),
            ('후보자별 득표수(득표율)',   '바른정당유승민'),
            ('후보자별 득표수(득표율)',    '정의당심상정'),
            ('후보자별 득표수(득표율)',   '새누리당조원진'),
            ('후보자별 득표수(득표율)',  '경제애국당오영국'),
            ('후보자별 득표수(득표율)', '국민대통합당장성민'),
            ('후보자별 득표수(득표율)', '늘푸른한국당이재오'),
            ('후보자별 득표수(득표율)',  '민중연합당김선동'),
            ('후보자별 득표수(득표율)',  '한국국민당이경희'),
            ('후보자별 득표수(득표율)',    '홍익당윤홍식'),
            ('후보자별 득표수(득표율)',    '무소속김민찬'),
            ('후보자별 득표수(득표율)',         '계'),
            (        '무효투표수',     '무효투표수'),
            (          '기권수',       '기권수')],
           )

In [30]:
df[0].index

RangeIndex(start=0, stop=26, step=1)

In [31]:
type(df[0].values[1:])

numpy.ndarray

In [32]:
df[0].values[1:][0]

array(['종로구', 133769, 102566, '42,512(41.59)', '22,325(21.84)',
       '22,313(21.83)', '7,412(7.25)', '7,113(6.95)', '228(0.22)',
       '5(0.00)', '78(0.07)', '31(0.03)', '63(0.06)', '26(0.02)',
       '47(0.04)', '49(0.04)', 102202, 364, 31203], dtype=object)

In [33]:
print(df[0].values[1:][0][0])
print(df[0].values[1:][0][2])
print(df[0].values[1:][0][3])
print(df[0].values[1:][0][4])
print(df[0].values[1:][0][5])

종로구
102566
42,512(41.59)
22,325(21.84)
22,313(21.83)


In [35]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append((each[2])
        data['moon'].append(get_num(str(each[3])))
        data['hong'].append(get_num(str(each[4])))
        data['ahn'].append(get_num(str(each[5])))

In [38]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        prev = {'광역시도':[], '시군':[], 'pop':[], 'moon':[], 'hong':[], 'ahn':[]}
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        pre['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(each[2])
        data['moon'].append(get_num(str(each[3])))
        data['hong'].append(get_num(str(each[4])))
        data['ahn'].append(get_num(str(each[5])))
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        pre['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
print(type(df[0].values[1:][0][2]))
print(type(df[0].values[1:][0][3]))

<class 'int'>
<class 'str'>


In [39]:
for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

In [40]:
election_result = pd.DataFrame(election_result_raw, 
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result

ValueError: arrays must all be same length

In [41]:
election_result_raw

{'광역시도': ['서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '서울특별시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '부산광역시',
  '대구광역시',
  '대구광역시',
  '대구광역시',
  '대구광역시',
  '대구광역시',
  '대구광역시',
  '대구광역시',
  '대구광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '인천광역시',
  '광주광역시',
  '광주광역시',
  '광주광역시',
  '광주광역시',
  '광주광역시',
  '대전광역시',
  '대전광역시',
  '대전광역시',
  '대전광역시',
  '대전광역시',
  '울산광역시',
  '울산광역시',
  '울산광역시',
  '울산광역시',
  '울산광역시',
  '세종특별자치시',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
  '경기도',
 

In [42]:
print(type(election_result_raw))

<class 'dict'>


In [43]:
election_result_raw.keys()

dict_keys(['광역시도', '시군', 'pop', 'moon', 'hong', 'ahn'])

In [44]:
election_result_raw['광역시도']

['서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '부산광역시',
 '대구광역시',
 '대구광역시',
 '대구광역시',
 '대구광역시',
 '대구광역시',
 '대구광역시',
 '대구광역시',
 '대구광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '인천광역시',
 '광주광역시',
 '광주광역시',
 '광주광역시',
 '광주광역시',
 '광주광역시',
 '대전광역시',
 '대전광역시',
 '대전광역시',
 '대전광역시',
 '대전광역시',
 '울산광역시',
 '울산광역시',
 '울산광역시',
 '울산광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도',
 '경기도'

In [45]:
print(type(election_result_raw['광역시도']))

<class 'list'>


In [46]:
for i in election_result_raw.keys():
    print(len(election_result_raw[i]))

251
251
250
250
250
250


In [47]:
for i in election_result_raw.keys():
    print(election_result_raw[i][249])
election_result_raw[election_result_raw.keys()[0]][250]
election_result_raw[election_result_raw.keys()[1]][250]

제주특별자치도
제주시
101296
43776.0
20036.0
21890.0


TypeError: 'dict_keys' object is not subscriptable

In [48]:
election_result_raw['광역시도'][250]
election_result_raw['시군'][250]

'서귀포시'

In [49]:
print(election_result_raw['광역시도'][250])
print(election_result_raw['시군'][250])

제주특별자치도
서귀포시


In [50]:
print(type(df[0].values[1:]))

<class 'numpy.ndarray'>


In [51]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        prev = {'광역시도':[], '시군':[], 'pop':[], 'moon':[], 'hong':[], 'ahn':[]}
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        pre['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(each[2])
        data['moon'].append(get_num(str(each[3])))
        data['hong'].append(get_num(str(each[4])))
        data['ahn'].append(get_num(str(each[5])))
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        prev['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        for k, v in prev.items():
            if v[0] == v[1]:
                print(f'{k} column has not been added')

In [52]:
for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

NameError: name 'pre' is not defined

In [53]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        prev = {'광역시도':[], '시군':[], 'pop':[], 'moon':[], 'hong':[], 'ahn':[]}
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        prev['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(each[2])
        data['moon'].append(get_num(str(each[3])))
        data['hong'].append(get_num(str(each[4])))
        data['ahn'].append(get_num(str(each[5])))
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        prev['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        for k, v in prev.items():
            if v[0] == v[1]:
                print(f'{k} column has not been added')

In [54]:
for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

NameError: name 'pre' is not defined

In [55]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        prev = {'광역시도':[], '시군':[], 'pop':[], 'moon':[], 'hong':[], 'ahn':[]}
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        prev['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(each[2])
        data['moon'].append(get_num(str(each[3])))
        data['hong'].append(get_num(str(each[4])))
        data['ahn'].append(get_num(str(each[5])))
        prev['광역시도'].append(len(data['광역시도']))
        prev['시군'].append(len(data['시군']))
        prev['pop'].append(len(data['pop']))
        prev['moon'].append(len(data['moon']))
        prev['hong'].append(len(data['hong']))
        prev['ahn'].append(len(data['ahn']))
        for k, v in prev.items():
            if v[0] == v[1]:
                print(f'{k} column has not been added')

In [56]:
for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

In [57]:
for i in election_result_raw.keys():
    print(len(election_result_raw[i]))

501
501
500
500
500
500


In [58]:
for i in election_result_raw.keys():
    election_result_raw[i] = election_result_raw[i][:250]

In [59]:
for i in election_result_raw.keys():
    print(len(election_result_raw[i]))

250
250
250
250
250
250


In [60]:
for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

In [61]:
for i in election_result_raw.keys():
    print(len(election_result_raw[i]))

500
500
500
500
500
500


In [62]:
for i in election_result_raw.keys():
    election_result_raw[i] = election_result_raw[i][250:500]

In [63]:
for i in election_result_raw.keys():
    print(len(election_result_raw[i]))

250
250
250
250
250
250


In [64]:
election_result = pd.DataFrame(election_result_raw, 
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result

,광역시도,시군,pop,moon,hong,ahn
0,서울특별시,종로구,102566,42512.0,22325.0,22313.0
1,서울특별시,중구,82852,34062.0,17901.0,19372.0
2,서울특별시,용산구,148157,58081.0,35230.0,32109.0
3,서울특별시,성동구,203175,86686.0,40566.0,45674.0
4,서울특별시,광진구,240030,105512.0,46368.0,52824.0
...,...,...,...,...,...,...
245,경상남도,산청군,24513,6561.0,12544.0,2753.0
246,경상남도,거창군,41325,11256.0,19976.0,4923.0
247,경상남도,합천군,33021,7143.0,19699.0,3077.0
248,제주특별자치도,제주시,273163,125717.0,48027.0,55971.0


In [65]:
election_result.to_csv('../data/05. election_result.csv', encoding='utf-8', sep=',')

FileNotFoundError: [Errno 2] No such file or directory: '../data/05. election_result.csv'

In [66]:
election_result.to_csv('../election_result.csv', encoding='utf-8', sep=',')

In [67]:
import os
print(os.getcwd())

C:\Users\bitcamp\pandas_prep
